In [ ]:
! pip install --quiet --upgrade --user google-cloud-aiplatform

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [1]:
# Define project information
PROJECT_ID = "your_google_cloud_project_id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.preview.vision_models import ImageGenerationModel

generation_model = ImageGenerationModel.from_pretrained("imagegeneration@006")

In [ ]:
!pip install flask-ngrok google-cloud-aiplatform google-cloud-storage
!pip install Flask
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("NGROK_API_KEY_HERE")

# Start Ngrok
public_url = ngrok.connect(5000)
print(" * Ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

In [ ]:
import datetime
import io
from flask import Flask, request, jsonify
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket('banner-generation-bigbasket')


app = Flask(__name__)

@app.route('/', methods=['POST'])
def generate_image():
    data = request.get_json()
    prompt = data.get('prompt', '')

    # Generate image using the prompt
    response = generation_model.generate_images(prompt=prompt)
    # response.images[0].show()
    # Save image to Google Cloud Storage
    image_blob = response.images[0]._pil_image
    img_byte_arr = io.BytesIO()
    image_blob.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Upload to Google Cloud Storage
    blob = bucket.blob(f'banners/generated_banner_{timestamp}.png') #change to where you want the files to be stored in the bucket
    try:
        blob.upload_from_file(img_byte_arr, content_type='image/png')
    except Exception as e:
        print("Error uploading to bucket:", e)
        return jsonify({'error': str(e)}), 500
    # Get public URL for the uploaded image
    public_url = f"https://storage.googleapis.com/google_cloud_bucket/banners/generated_banner_{timestamp}.png"  #change according to your bucket name

    return jsonify({'image_url': public_url})

if __name__ == '__main__':
    app.run(port=5000)  # Specify the port to run on
!flask run --host=0.0.0.0 --port=5000

In [ ]:
#run in a separate notebook to verify server running
import requests

# Replace with your Ngrok URL
ngrok_url = 'Ngrok URL'  # e.g., http://abc123.ngrok.io

# Send a POST request to the Flask backend
data = {'prompt': 'A beautiful sunset over the mountains.'}
response = requests.post(ngrok_url, json=data)

# Print the response
print(response.text)